In [1]:
from sqlalchemy import create_engine
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split #train
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report #check
from sklearn.model_selection import cross_val_score
from collections import Counter
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [2]:
#parameters to connect to database
db_host="18.136.56.185"
Username="dm_team3"
Password="dm_team15119#"
db_Name="project_telecom"

In [3]:
conn=create_engine('mysql+pymysql://'+Username+':'+Password+'@'+db_host+'/'+db_Name)
conn.table_names()

['telecom_churn_data']

In [4]:
query = "select * from telecom_churn_data"
orders = pd.read_sql(query,conn) #extracting the table
print(orders.shape)
print(orders.head())

(4617, 21)
  columns1 columns2 columns3   columns4 columns5 columns6 columns7 columns8  \
0       KS      128      415   382-4657       no      yes       25    265.1   
1       OH      107      415   371-7191       no      yes       26    161.6   
2       NJ      137      415   358-1921       no       no        0    243.4   
3       OH       84      408   375-9999      yes       no        0    299.4   
4       OK       75      415   330-6626      yes       no        0    166.7   

  columns9 columns10  ... columns12 columns13 columns14 columns15 columns16  \
0      110     45.07  ...        99     16.78     244.7        91     11.01   
1      123     27.47  ...       103     16.62     254.4       103     11.45   
2      114     41.38  ...       110      10.3     162.6       104      7.32   
3       71      50.9  ...        88      5.26     196.9        89      8.86   
4      113     28.34  ...       122     12.61     186.9       121      8.41   

  columns17 columns18 columns19 columns

In [5]:
orders.head()

,columns1,columns2,columns3,columns4,columns5,columns6,columns7,columns8,columns9,columns10,...,columns12,columns13,columns14,columns15,columns16,columns17,columns18,columns19,columns20,columns21
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10,3,2.7,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.7,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.3,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.9,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


In [6]:
#Tacking the column name in variable 'a' to update the dataframe with these column
a="State,Account,Area,Phone,International_Plan,VMail_Plan,VMail_Message,Day_Mins,Day_Calls,Day_Charge,Eve_Mins,Eve_Calls,Eve_Charge,Night_Mins,Night Calls,Night_Charge,International_Mins,International_calls,International_Charge,CustServ_Calls,Churn"
columns=a.split(",") #column name in column variable in form of list

In [7]:
#updating the column of Orders
orders.columns=columns 

In [8]:
orders.head()

,State,Account,Area,Phone,International_Plan,VMail_Plan,VMail_Message,Day_Mins,Day_Calls,Day_Charge,...,Eve_Calls,Eve_Charge,Night_Mins,Night Calls,Night_Charge,International_Mins,International_calls,International_Charge,CustServ_Calls,Churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10,3,2.7,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.7,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.3,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.9,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


## Encoding


In [9]:
#As some of the column contain 'yes','no' their value, we need to conver it into numerical form 
#before giving these data to our model
# 0 means No and 1 means Yes
enc=LabelEncoder()
X=orders.iloc[:,:]
X.International_Plan=enc.fit_transform(X.International_Plan)
X.VMail_Plan=enc.fit_transform(X.VMail_Plan)
X.Churn=enc.fit_transform(X.Churn)

## basic Check

In [10]:
X.duplicated().sum()

0

In [11]:
X.isnull().sum()

State                   0
Account                 0
Area                    0
Phone                   0
International_Plan      0
VMail_Plan              0
VMail_Message           0
Day_Mins                0
Day_Calls               0
Day_Charge              0
Eve_Mins                0
Eve_Calls               0
Eve_Charge              0
Night_Mins              0
Night Calls             0
Night_Charge            0
International_Mins      0
International_calls     0
International_Charge    0
CustServ_Calls          0
Churn                   0
dtype: int64

## converting str to numeric in data frame


In [12]:
#some of the column contain numerical data but in string form, so wee need to convert those string number to numerical form
for i in columns:
    try:
        X[i]=pd.to_numeric(X[i])
    except:
        pass
print("done")

done


In [13]:
X.head()

,State,Account,Area,Phone,International_Plan,VMail_Plan,VMail_Message,Day_Mins,Day_Calls,Day_Charge,...,Eve_Calls,Eve_Charge,Night_Mins,Night Calls,Night_Charge,International_Mins,International_calls,International_Charge,CustServ_Calls,Churn
0,KS,128,415,382-4657,0,1,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,OH,107,415,371-7191,0,1,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,NJ,137,415,358-1921,0,0,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,OH,84,408,375-9999,1,0,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,OK,75,415,330-6626,1,0,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


## target and X


In [14]:
y=X.Churn
X=X.iloc[:,:-1]
print(y.shape)
print(X.shape)
print(Counter(y))

(4617,)
(4617, 20)
Counter({0: 3961, 1: 656})


## not taking first 4 column
- state
- account
- area
- phone

In [15]:
#not taking these 4 columns as they are not requried for training model
#account can be considred while trainig the model but while checking recall value with account column, I got less accuracy so 
# I removed it 
X=X.iloc[:,4:]
X.head()

,International_Plan,VMail_Plan,VMail_Message,Day_Mins,Day_Calls,Day_Charge,Eve_Mins,Eve_Calls,Eve_Charge,Night_Mins,Night Calls,Night_Charge,International_Mins,International_calls,International_Charge,CustServ_Calls
0,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3


## Under Sampling

In [16]:
# As this is Imbalance data set we need to balanced it.
# There are NO(0):3961, YES(1):656 of churn record
# Under Sampling is good here as we are more intrested in YES(1), so removing some of 0's is good idea to make the data set as
#Balanced

from imblearn.under_sampling import NearMiss
nm=NearMiss()                   #creating object
X_under,y_under=nm.fit_sample(X,y)  #under sampling
print(X_under.shape)     
X_under.head()
X_under['churn']=y_under            #putting back y_new to X_new to shuffle the data
X_under=X_under.sample(frac=1).reset_index(drop=True)  #shuffling data

(1312, 16)


## Again creating target and Features From under sample data

In [17]:
y=X_under.churn
xx=X_under.iloc[:,:-1]
print(y.shape)
print(xx.shape)
print(Counter(y))

(1312,)
(1312, 16)
Counter({1: 656, 0: 656})


## Deleting columns with high co-relation

In [18]:
d=['VMail_Plan',
 'Day_Charge',
 'Eve_Charge',
 'Night_Charge',
 'International_Charge']
del xx['Day_Charge']
del xx['Eve_Charge']
del xx['Night_Charge']
del xx['International_Charge']
del xx['VMail_Plan']

## train test split


In [19]:
X_train,X_test,y_train,y_test=train_test_split(xx,y,test_size=0.3,random_state=5)


## counter gives the number of taget avaliable

In [20]:
print("y_train"+str(Counter(y_train)))
print("y_test"+str(Counter(y_test)))

y_trainCounter({1: 465, 0: 453})
y_testCounter({0: 203, 1: 191})


In [21]:
xx.shape

(1312, 11)

## XGBoost

In [38]:
model_xgb=XGBClassifier(n_estimators=156,min_child_weight=2,gamma=0.35,learning_rate=0.2,colsample_bytree=0.7,booster='gbtree')
model_xgb.fit(X_train,y_train)
##n_estimators=70,min_child_weight=2,gamma=0.35,learning_rate=0.2,colsample_bytree=0.7,booster='gbtree'

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.35,
              learning_rate=0.2, max_delta_step=0, max_depth=3,
              min_child_weight=2, missing=None, n_estimators=156, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [39]:
y_predict=model_xgb.predict(X_test)


In [40]:
accuracy_score(y_test,y_predict)

0.8908629441624365

In [41]:
print(Counter(y_test))
pd.crosstab(y_test,y_predict)


Counter({0: 203, 1: 191})


col_0,0,1
churn,,
0,179,24
1,19,172


In [42]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89       203
           1       0.88      0.90      0.89       191

    accuracy                           0.89       394
   macro avg       0.89      0.89      0.89       394
weighted avg       0.89      0.89      0.89       394



## K fold validation

In [43]:
cv_recall=cross_val_score(model_xgb,xx,y,cv=10,scoring='recall')
print(cv_recall)
print(cv_recall.mean())

[0.83333333 0.89393939 0.90909091 0.83333333 0.84848485 0.86363636
 0.92307692 0.86153846 0.92307692 0.86153846]
0.8751048951048952


## Project end here. Below work is rough or me just experimenting 

## Stratified K fold

In [28]:
from sklearn.model_selection import StratifiedKFold

In [225]:
accuracy=[]
skf=StratifiedKFold(n_splits=10)
#skf.get_n_split(xx,y)
for train_index,test_index in skf.split(xx,y):
    x1_train,x1_test=xx.iloc[train_index],xx.iloc[test_index]
    y1_train,y1_test=y.iloc[train_index],y.iloc[test_index]
    model_xgb.fit(x1_train,y1_train)
    y_predict=model_xgb.predict(x1_test)
    score=accuracy_score(y1_test,y_predict)
    accuracy.append(score)
print(accuracy)
print(sum(accuracy)/len(accuracy))

[0.8939393939393939, 0.9015151515151515, 0.916030534351145, 0.8244274809160306, 0.8854961832061069, 0.9007633587786259, 0.8931297709923665, 0.916030534351145, 0.9083969465648855, 0.9083969465648855]
0.8948126301179735


## tuning hyper parameters

In [68]:

parameters=[{'n_estimators':range(100,160),
             'gamma':[0.35,0.4,0.5],
             'booster':['gbtree'],
             'learning_rate':[0.20,0.25,0.3],
            'min_child_weight':[2,3,4,7],
            'colsample_bytree':[0.6,0.65,0.7,0.75]}]
grid_search=GridSearchCV(estimator=model_xgb,param_grid=parameters,cv=10,n_jobs=-1)
grid_search.fit(xx,y)
print(grid_search.best_score_)
print(grid_search.best_params_)

## RandomizedSearchCV

In [63]:
from sklearn.model_selection import RandomizedSearchCV

In [72]:
for i in range(12):
    search=RandomizedSearchCV(estimator=model_xgb,param_distributions=parameters,n_iter=1,cv=10,scoring='recall')
    search.fit(xx,y)
    p=search.best_params_
    s=search.best_score_
    print(p)
    print(s)

{'n_estimators': 156, 'min_child_weight': 2, 'learning_rate': 0.2, 'gamma': 0.35, 'colsample_bytree': 0.7, 'booster': 'gbtree'}
0.8872494172494173


KeyboardInterrupt: 

In [451]:
X_train,X_test,y_train,y_test=train_test_split(xx,y,test_size=0.3,random_state=5)

In [111]:
y=X_new['churn']

In [115]:
xx=X_new.iloc[:,:-1]

In [473]:
xx.head()

,International_Plan,VMail_Message,Day_Mins,Day_Calls,Eve_Mins,Eve_Calls,Night_Mins,Night Calls,International_Mins,International_calls,CustServ_Calls
0,0,0,157.7,101,298.6,100,216.9,99,13.8,3,1
1,0,0,93.5,110,213.3,79,171.7,83,8.9,4,4
2,0,0,247.7,77,269.5,108,167.3,82,9.6,7,0
3,0,0,171.6,119,205.0,107,170.6,114,13.8,4,0
4,1,0,157.8,71,203.2,114,168.7,82,10.0,2,3


## Churn Flag

In [242]:
d=[{'phone':9564712077,'International_Plan':0,'VMail_Message':0,'Day_Mins':10,
   'Day_Calls':2,'Eve_Mins':0,'Eve_Calls':0,'Night_Mins':5,
   'Night Calls':1,'International_Mins':0,'International_calls':0,
   'CustServ_Calls':6},
   
   {'phone':7602162477,'International_Plan':0,'VMail_Message':0,'Day_Mins':157.7,
   'Day_Calls':101,'Eve_Mins':298.6,'Eve_Calls':100,'Night_Mins':216.8,
   'Night Calls':99,'International_Mins':13.8,'International_calls':3,
   'CustServ_Calls':1}]
testdf=pd.DataFrame(d)

In [243]:
phone=[]
phone.append(testdf.phone)

In [244]:
phone

[0    9564712077
 1    7602162477
 Name: phone, dtype: int64]

In [245]:
del testdf['phone']

In [246]:
churn_flag=model_xgb.predict(testdf) #Predcting with XGBoost

In [247]:
phonedf=pd.DataFrame(phone[0]) #Creating Phone DataFrame

In [248]:
churndf=pd.DataFrame(p) #Creating Churn FLag DataFrame

In [249]:
churn_DF=phonedf.join(churndf)

In [250]:
churn_DF.columns=['PHONE','CHURN_FLAG']

In [251]:
churn_DF

,PHONE,CHURN_FLAG
0,9564712077,1
1,7602162477,0


In [253]:
for i in range(len(churn_DF.CHURN_FLAG)):
    churn_DF.CHURN_FLAG[i]='YES' if churn_DF.CHURN_FLAG[i]==1 else 'NO'

C:\Users\gaurav\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\gaurav\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [254]:
churn_DF

,PHONE,CHURN_FLAG
0,9564712077,YES
1,7602162477,NO


In [255]:
from sklearn.model_selection import cross_val_predict

In [256]:
y_pre=cross_val_predict(model_xgb,xx,y,cv=10)

In [258]:
print(Counter(y))
pd.crosstab(y,y_pre)

Counter({1: 656, 0: 656})


col_0,0,1
churn,,
0,592,64
1,74,582
